In [2]:
import requests
import threading
import pymongo
import random
import logging
import json
import time
from queue import Queue
from bs4 import BeautifulSoup

In [1]:
class Mongo():
    
    def __init__(self):
        self._client = pymongo.MongoClient("mongodb://localhost:27017/")
        self._db = None
        self._collection = None
    
    def insert(self, doc):
        if type(doc) is dict:
            result = self._collection.insert_one(doc)
        else:
            result = self._collection.insert_many(doc)
        return result

    def get_collection(self):
        return self._collection
    
    def find_one(self, doc=None):
        if doc is not None:
            result = self._collection.find_one(doc)
        else:
            result = self._collection.find_one()
        return result
    
    def find(self, doc=None):
        if doc is not None:
            result = self._collection.find(doc)
        else:
            result = self._collection.find()
        return result
    
    def count(self, doc=None):
        if doc is not None:
            count = self._collection.find(doc).count()
        else:
            count = self._collection.find().count()
        return count

In [6]:
class MongoDouban(Mongo):
    __instance_lock = threading.Lock()
    __init_flag = False
    
    def __init__(self):
        if self.__init_flag is False:
            super(MongoDouban, self).__init__()
            self._db = self._client["douban"]
            self.__init_flag = True
    
    def __new__(cls, *args, **kwargs):
        if not hasattr(MongoDouban, "_instance"):
            with MongoDouban.__instance_lock:
                if not hasattr(MongoDouban, "_instance"): 
                    MongoDouban._instance = object.__new__(cls)
        return MongoDouban._instance

In [7]:
class DoubanMovieCollection(MongoDouban):
    
    def __init__(self):
        super(DoubanMovieCollection, self).__init__()
        self._collection = self._db['movie']

In [54]:
class MongoDouban():
    __instance_lock = threading.Lock()
    __init_flag = False
    _db = None
    _collection = None
    
    def __init__(self):
        if self.__init_flag is False:
            print('init')
            mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
            self._db = mongo_client["douban"]
            self.__init_flag = True
    
    def __new__(cls, *args, **kwargs):
        if not hasattr(MongoDouban, "_instance"):
            with MongoDouban.__instance_lock:
                if not hasattr(MongoDouban, "_instance"): 
                    MongoDouban._instance = object.__new__(cls)
        return MongoDouban._instance
    
    def insert(self, doc):
        if type(doc) is dict:
            result = self._collection.insert_one(doc)
        else:
            result = self._collection.insert_many(doc)
        return result

    def get_collection(self):
        return self._collection
    
    def find_one(self, doc=None):
        if doc is not None:
            result = self._collection.find_one(doc)
        else:
            result = self._collection.find_one()
        return result
    
    def find(self, doc=None):
        if doc is not None:
            result = self._collection.find(doc)
        else:
            result = self._collection.find()
        return result
    
    def count(self, doc=None):
        if doc is not None:
            count = self._collection.find(doc).count()
        else:
            count = self._collection.find().count()
        return count

In [13]:
def run_time(func):
    def wrapper(*args, **kw):
        start = time.time()
        func(*args, **kw)
        end = time.time()
        print('running', end-start, 's')
    return wrapper

In [22]:
class UserAgent():
    __instance_lock = threading.Lock()
    __init_flag = False
    
    def __init__(self):
        if self.__init_flag is False:
            print('init UserAgent')
            self.__agents_pool = list()
            with open('./UserAgents/useragents.txt','r') as read_ob:
                for line in read_ob.readlines():
                    self.__agents_pool.append(line.strip())
            self.__init_flag = True
    
    def __new__(cls, *args, **kwargs):
        if not hasattr(UserAgent, "_instance"):
            print('new UserAgent')
            with UserAgent.__instance_lock:
                if not hasattr(UserAgent, "_instance"): 
                    UserAgent._instance = object.__new__(cls)
        return UserAgent._instance
    
    def get_useragent_randomly(self):
        return random.choice(self.__agents_pool)
    

In [23]:
class Crawl():

    def __init__(self):
        self._session = None
        self._headers = {
            'User-Agent':UserAgent().get_useragent_randomly(),
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language':'zh-cn',
            'Connection': 'keep-alive',
            'Accept-Encoding': 'gzip, deflate',
        }
        self._proxies = [
            {"http":"112.111.217.114:9999"},
            {"http":"180.118.128.118:9000"},
            {"http":"171.11.29.217:9999"},
            {"http":"120.83.109.191:9999"}
        ]
        
    def request_get(self, url, **kwargs):
        logging.info('scraping {}...'.format(url))
        try:
            response = requests.get(url, headers=self._headers, proxies=random.choice(self._proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url))
     
    def request_post(self, url, **kwargs):
        logging.info('scraping {}...'.format(url))
        try:
            response = requests.post(url, headers=self._headers, proxies=random.choice(self._proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url))
    
    def session_get(self, url, **kwargs):
        self.check_session()
        logging.info('scraping {}...'.format(url))
        try:
            response = self._session.get(url, headers=self._headers, proxies=random.choice(self._proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url))
    
    def session_post(self, url, **kwargs):
        self.check_session()
        logging.info('scraping {}...'.format(url))
        try:
            response = self._session.post(url, headers=self._headers, proxies=random.choice(self._proxies), **kwargs)
            if response.status_code == 200:
                return response
            logging.error('get invalid status code %s while scraping %s', response.status_code, url)
        except requests.RequestException:
            logging.error('error occurred while scraping %s', url, exc_info=True)
        else:
            logging.info('scraping {} finished'.format(url)) 

    def get_session(self):
        return self._session
    
    def set_session(self, session):
        self._session = session
        return True
    
    def save_cookies(self, user):
        cookieJar = requests.cookies.RequestsCookieJar()
        for cookie in user.cookies:
            cookieJar.set(cookie.name,cookie.value)
        for cookie in user.headers['Set-Cookie'].split(";"):
            key = cookie.split('=')[0]
            value = cookie.split('=')[1]
            cookieJar.set(key,value)
        return cookieJar
    
    def check_session(self):
        if self._session == None:
            self._session = requests.session()
        return
    
    def add_header(self, headers):
        for key, value in headers.items():
            self._headers[key] = value


In [16]:
class DoubanUser(Crawl):
    __instance_lock = threading.Lock()
    __init_flag = False
       
    def __init__(self):
        if self.__init_flag is False:
            super(DoubanUser, self).__init__()
            self.__user = None
    #         self._cookies = None
            self.__login()
            self.__init_flag = True

    def __new__(cls, *args, **kwargs):
        if not hasattr(DoubanUser, "_instance"):
            print('new Douban')
            with DoubanUser.__instance_lock:
                if not hasattr(Douban, "_instance"): 
                    DoubanUser._instance = object.__new__(cls)
        return DoubanUser._instance
    
    def __login(self):
        post_data = {
            'name':'18664678368',
            'password':'Ljc970412',
            'remember':'false'
        }
        self.add_header({
            "Referer":'https://accounts.douban.com/passport/login'
        })
        user = self.session_post('https://accounts.douban.com/j/mobile/login/basic', data=post_data)
        login_detail = json.loads(user.text)
        if login_detail['status'] == 'success':
            print('login success!')
#             self.__cookies = self.save_cookies(user)
        else:
            print('login failed!')
        self.__user =  user.text
    
    def get_user_info(self):
        return self.__user

In [48]:
class Douban(Crawl):
    
    def __init__(self, douban_user):
        super(Douban, self).__init__()
        self.set_session(douban_user.get_session())
    
    def search(self, query, cat=''):
        res = dict()
        params = {
            'q':query,
            'cat':cat
        }
        count=0
        response = self.session_get('https://www.douban.com/search', params=params)
        soup = BeautifulSoup(response.content, 'html.parser')
        results = soup.find(class_='result-list').find_all(class_='result')
        for result in results:
            pic = result.find(class_='pic')
            content = result.find(class_='content')
            img = pic.img.get('src')
            link = content.a.get('href')
            name = content.a.text
            description = content.p.text if content.p else ''
            res[count]={
                'name':name,
                'img':img,
                'link':link,
                'description':description
            }
            count+=1
        return res

In [207]:
class DoubanMovie(Douban):
    
    
    def get_nowshowing_movies(self):
        response = self.request_get('https://movie.douban.com/cinema/nowplaying')
        soup = BeautifulSoup(response.text)
        nowshowing_movies = soup.find(class_='lists').find_all(class_='list-item')
        movies_list = []
        for nowshowing_movie in nowshowing_movies:
            movie = {
                'movie_id':nowshowing_movie.attrs['id'],
                'title':nowshowing_movie.attrs['data-title'],
                'actors':nowshowing_movie.attrs['data-actors'],
                'director':nowshowing_movie.attrs['data-director'],
                'score':nowshowing_movie.attrs['data-score'],
                'release':nowshowing_movie.attrs['data-release'],
                'region':nowshowing_movie.attrs['data-region']
            }
            movies_list.append(movie)
            print(movie)
        return movies_list
    
    def get_movie_info(self, url):
        response = self.request_get(url)
        soup = BeautifulSoup(response.text)
        
        movie = {
            'movie_id':re.findall(r'https://movie.douban.com/subject/([0-9]*)',url)[0],
            'title':soup.find(property="v:itemreviewed").text.split()[0],
            'score':soup.find(property="v:average").text,
            'release':re.findall(r'[(](.*?)[)]', soup.find(class_="year").text)[0]
        }
        for info in soup.find(id='info').text.split('\n'):
            if '导演' in info:
                movie['director'] = info.split(': ')[1]
            elif '主演' in info:
                movie['actors'] = info.split(': ')[1]
            elif '国家' in info:
                movie['region'] = info.split(': ')[1]
            elif '主演' in info:
                movie['actors'] = info.split(': ')[1]
        print(movie)
        return movie

    def get_top_250(self):
        pass
    
    
    def get_recently_hot_movie(self, **kwargs):
        pass
    
    @run_time
    def get_comments(self, **kwargs):
        
        count = 0
        queue = Queue()
        producer_running = True
        
        def producer(url=None, movie_id=None, page_max=50, page=0):
            nonlocal producer_running
            if producer_running is True and page <= page_max:
                movie_url = None
                if movie_id is not None:
                    movie_url = 'https://movie.douban.com/subject/{}/comments'.format(str(movie_id))
                else:
                    movie_url = url
                print("开始爬取第{0}页***********************************************************************：".format(page+1))
                params={
                    'start':page*20,
                    'limit':20,
                    'status':'P',
                    'sort':'new_score'
                }
                html = self.session_get(movie_url, params=params)
                soup = BeautifulSoup(html.content, 'html.parser')
                comments = soup.find(id='comments').find_all(class_='comment-item')
                if len(comments) > 1:
                    queue.put(comments)
                    producer(url=movie_url, page=page+1, page_max=page_max)
                else:
                    producer_running=False
            else:
                producer_running=False
            
        def customer():
            nonlocal count
            while producer_running is True or queue.empty() is False:
                comments = queue.get()
                for comment in comments:
                    content = comment.find(class_='comment-content').text
                    user_name = comment.find(class_='comment-info').a.text
                    comment_time = comment.find(class_='comment-info').find(class_='comment-time').attrs['title']
                    
                    print('comment:{}, user_name:{}, comment_time:{}'.format(content,user_name,comment_time))
                    count += 1
                print("count={}".format(count))
                time.sleep(int(random.choice([0.5, 0.2, 0.3])))
                
        threads = list()
        
        thread_p = threading.Thread(target=producer, kwargs=kwargs)
        thread_p.start()
        threads.append(thread_p)
        
        for _ in range(10):
            thread_c = threading.Thread(target=customer)
            thread_c.start()
            threads.append(thread_c)
            
        for thread in threads:
            thread.join(timeout=0.5)

In [46]:
class DoubanMovie(Douban):
    
    @run_time
    def get_comment(self, url=None, movie_id=None, page_max=50):
        movie_url = None
#         if movie_name is not None:
            
        if movie_id is not None:
            movie_url = 'https://movie.douban.com/subject/{}/comments'.format(str(movie_id))
        else:
            movie_url = url
        count = 0
        for page in range(page_max+1):
            print("开始爬取第{0}页***********************************************************************：".format(page+1))
            params={
                'start':page*20,
                'limit':20,
                'status':'P',
                'sort':'new_score'
            }
            html = self.session_get(movie_url, params=params)
            print(html.url)
            soup = BeautifulSoup(html.content, 'html.parser')
            comments = soup.find(id='comments').find_all(class_='comment-item')
            if len(comments) > 1:
                for comment in comments:
                    content = comment.find(class_='comment-content').text
                    user_name = comment.find(class_='comment-info').a.text
                    comment_time = comment.find(class_='comment-info').find(class_='comment-time').attrs['title']
                    print('comment:{}, user_name:{}, comment_time:{}'.format(content,user_name,comment_time))
                    count += 1
                print("count={}".format(count))
                time.sleep(int(random.choice([0.5, 0.2, 0.3])))
            else:
                print("大约共{0}页评论".format(page+1))
                break
            
#     def get_review_by_url(self, url , page_max=50):
#         res = dict()
#         self.set_url(url)
#         count = 0
#         for page in range(page_max+1):
#             print("开始爬取第{0}页***********************************************************************：".format(page+1))
#             self.set_params({
#                 'start':page*20
#             })
#             html = self.session_get()
#             soup = BeautifulSoup(html.content, 'html.parser')
#             reviews = soup.find(class_='review-list').find_all(class_='review-item')
#             for review in reviews:
#                 user = review.find(class_='main-hd')
#                 user_name = user.find(class_='name').text
#                 user_link = user.find(class_='name').a.get(href)
                
#                 content = review.find(class_='main-bd')
#                 title = content.find('h2').text
#                 link = content.find('h2').a.get('href')
#                 self.set_url(link)
#                 review_response = self.request_get()
#                 review_soup = BeautifulSoup(review_response, 'html.parser')
#                 article = review_soup.find(class_='article')
#                 review_content = article.find(class_='review-content').text
#                 res[title] = {
#                     'user_name':user_name,
#                     'user_link':user_link,
#                     'link':link,
#                     'review_content':review_content
#                 }
#             return res
        
#     def get_review_by_id(self, movie_id, page_max=50):
#         url = 'https://movie.douban.com/subject/{}/reviews'.format(str(movie_id))
#         self.get_review_by_url(url, page_max)
    
    def search(self, movie_name):
        result = super().search(movie_name, cat='1002')
        return result